## SCRAPE Code 'Unlisted Companies in India' from the .json file created post CRAWL

## Install and Import libraries

In [ ]:
# !pip install bs4
# !pip install lxml
# !pip install requests_html
import csv
import json
import urllib
from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup
import time
from requests_html import HTMLSession

## Open CSV file to append data, generate dictionary key value pairs (basis index number) in order to append Columns from the website to be scraped.

In [ ]:
csvfile = open('test.csv', 'w', newline='',encoding="utf-8")

make_table = csv.writer(csvfile, delimiter = ';')

attribute_to_element = {
    "CIN": ["td"],
    "Company Status": ["td"],
    "RoC": ["td"],
    "Registration Number": ["td"],
    "Company Category": ["td"],
    "Company Subcategory": ["td"],
    "Class of Company": ["td"],
    "Date of Incorporation": ["td"],
    "Age of Company": ["td"],
    "Activity": ["td"],
    "Number of Members": ["td"],
    "Authorized Capital": ["td"],
    "Paid up Capital": ["td"],
    "Date of Last Annual General Meeting": ["td"],
    "Date of Latest Balance Sheet": ["td"],
    "Email ID" : ["p"],
    "Website" : ["p"],
    "Address" : ["p"],
    "Number of Directors" : ["table"],
    "No of Prosecution" : ["table"],
    "Charges/Borrowing Details" : ["table"]
    
}
attribute_indexes = {
    "CIN": 1,
    "Company Status": 5,
    "RoC": 7,
    "Registration Number": 9,
    "Company Category": 11,
    "Company Subcategory": 13,
    "Class of Company": 15,
    "Date of Incorporation": 17,
    "Age of Company": 19,
    "Activity": 21,
    "Number of Members": 23,
    "Authorized Capital": 27,
    "Paid up Capital": 29,
    "Date of Last Annual General Meeting": 35,
    "Date of Latest Balance Sheet": 37,
    "Email ID" : 61,
    "Website" : 62,
    "Address" : 64,
    "Number of Directors" : 7,
    "No of Prosecution" : 10,
    "Charges/Borrowing Details" : 11
}
all_attributes = ["Name", "URL"]


## Run scraping on a single link by setting test_out = True
## Run scraping on a Crawled_URLs.json file by setting test_out = False

In [ ]:
for attribute in attribute_to_element.keys():
    all_attributes.append(attribute)

make_table.writerow(all_attributes)

test_out = False

if test_out:
    links = ['https://www.zaubacorp.com/company/GHODWADI-MOVIES-PRIVATE-LIMITED/U22222MH2012PTC232389']
else:
    with open('crawled_URLs.json', 'r+') as f:
        links = json.load(f)

    print('Number of raw URLs to look at: ' + str(len(links)))

agent = "Safari"
# Reference: https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent

view_url = True
view_count = True


## Use the Crawler based on Beautiful Soup library to extract content from the desired Web page.

In [ ]:
def add_element(listed, item):
    # process and add item to listed
    # print(item)
    if item:
        text = item.text.replace("\n", "")
        listed.append(text)
    else:
        listed.append("not_available")

    return listed

def handle_element(soup, element, key):
    return soup.find_all(element)[attribute_indexes[key]]

for url in links:
    time.sleep(1)
    if 'https://www.zaubacorp.com/company/' not in url:
        continue
    if view_url:
        print('URL: ' + url, flush = True)

    try:

        html = urlopen(Request(url, headers = {'User-Agent': agent})).read()
        print("ok")
        # html = requests.get(url).text
    
    except Exception as E:
        print(url + ' not found.', flush = True)
        continue

    soup = BeautifulSoup(html, 'lxml')

    listed = []
    
    # name
    setup = soup.find("h1")

    # Also checks if it is a valid page
    if setup:
        listed.append((setup.text).strip())
    else:
        # Not a valid page
        continue
    
    # url
    listed.append(url)

    ctr2 = 0
    for key in attribute_to_element.keys():

        element = attribute_to_element[key]
        setup = handle_element(soup, element, key)
        
        ctr1 = 0
        
        
        if key== "Number of Directors":
            links = setup.find_all("a")
            for l in links:
              if l.text.strip() == "View other directorships":
                ctr2+=1
            # ctr2 = len(setup.find_all("a"))//6
            listed.append(str(ctr2))
            #print(ctr2)
        elif key== "No of Prosecution":
            
            setup = soup.find_all(element)[attribute_indexes[key]+(2*ctr2)]
            prosecution_list = (len(setup.find_all('td'))//6)-1
            listed.append(str(prosecution_list))
        
        
        elif key== "Charges/Borrowing Details":
            
            setup = soup.find_all(element)[attribute_indexes[key]+(2*ctr2)]
            charges_borrowing = setup.find_all('td')
            
            
            try:
                i=12
                while True:
                                       
                    ctr1 += float(charges_borrowing[i].text.replace(",",""))
                    
                    i += 7
            except:
                listed.append(str(ctr1))
            
        
            
        elif key== "Email ID" or key=="Website":
            text = setup.text[setup.text.find(":")+2:]
            text = text.replace("\n", "")
            listed.append(text)
            
        else:
            listed = add_element(listed, setup)
    
    
    try:
    
        make_table.writerow(listed)
        csvfile.flush()
        print(listed)
    except:
        pass

## Make Excel and .Json uncleaned output.

In [ ]:

import pandas as pd
 
 
# Reading the csv file
df_new = pd.read_csv('test.csv', sep = ';')
 
# saving xlsx file
xlfile = pd.ExcelWriter('Companies_final.xls')
df_new.to_excel(xlfile, index=False)
xlfile.save()
xlfile.close()

In [ ]:

# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf, delimiter=';')
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'CIN' to be the primary key
            
            #print(rows)
            
            key = rows['CIN']
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
        
make_json('test.csv','final_uncleaned.json')